
<br> 
# <span style="font-family:Didot; font-size:3em;"> Parikshit Gupta </span>


<a href="https://github.com/parikshitgupta1/mask-detection/blob/master/4_Train_Model.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



# Tensorflow - Objection Detection API Training#

<img src="https://miro.medium.com/max/128/1*cKG1LJvVTaWqSkYSyVqtsQ.png" alt="Visit my Blog">

We are now ready to get our hands dirty working directly with Tensorflow!

We have one final piece of data preperation to do which is create TF Records then we can move to training our model!

At the time of writing, the Tensorflow Object Detection API did not support v2 of the Tensorflow framework therefore we will be working with v1.15

# Setup

## Confirm GPU is present

In [ ]:
!nvidia-smi

## Install required libraries
Be sure to click on the **restart** button after running this cell

In [ ]:
!pip install -U --pre tensorflow-gpu=="1.*"
!pip install tf_slim
!apt-get update && apt-get upgrade -y
!pip install numpy==1.16 #This version is required due to a bug. After installing this version, the runtime has to be restarted.

Make sure you have `pycocotools` installed

In [ ]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
!apt-get install python-pil python-lxml 

!pip install Cython
!pip install contextlib2
!pip install matplotlib
!pip install wget

import os
import pathlib
import wget


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

os.environ['PYTHONPATH'] += ":/content/models/research/:/content/models/research/slim"

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd /content/models/research
pip install .

In [ ]:
%%bash
cd /content/models/research/object_detection/builders/
python model_builder_test.py

# Imports & Configuration

In [ ]:
!wget https://raw.githubusercontent.com/parikshitgupta1/mask-detection/master/config/constants.py
from constants import *

In [ ]:
!curl {PRETRAINED_MODEL_CONFIG} -o {CONFIG_DIR}pipeline.config
wget.download(LABEL_MAP,DATASET_DIR_PREP_RECORD)
wget.download(TF_RECORD,PRE_PROCESS_DIR)
%cd {TF_MODEL_DIR}
!wget {PRETRAINED_MODEL_URL}
!tar -xzf {PRETRAINED_MODEL_NAME}.tar.gz

## Connect to Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

## Extract dataset ###

In [ ]:
%cd {DRIVE_DEV}
%cp part3-datasetv6.zip /content/
%cd /
!unzip -o /content/part3-datasetv6.zip

# Prepare TFRecord Files ##

Now that we have generated our annotations and split our dataset into the desired training and testing subsets, it is time to convert our annotations into the so called TFRecord format.

Most of the batch operations aren’t done directly from images, rather they are converted into a single tfrecord file (images which are numpy arrays and labels which are a list of strings).

## Generate the TF Records
Amend paths as 

In [ ]:
%cd {PRE_PROCESS_DIR}
!python generate_tfrecord.py --csv_input="{DATASET_DIR_PREP_RECORD}/train_labels.csv" --output_path="{DATASET_DIR_PREP_RECORD}/train.record" --img_path="{DATASET_DIR_PREP_TRAIN_IMG}" --label_map="{DATASET_DIR_PREP_RECORD}/label_map.pbtxt"
!python generate_tfrecord.py --csv_input="{DATASET_DIR_PREP_RECORD}/test_labels.csv" --output_path="{DATASET_DIR_PREP_RECORD}/test.record" --img_path="{DATASET_DIR_PREP_TEST_IMG}" --label_map="{DATASET_DIR_PREP_RECORD}/label_map.pbtxt"

# Train

## Check Remaining time

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

## Start Tensorboard

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir={CKPT_DIR}/

### Refresh pipeline config (when developing)

In [ ]:
!rm -r {CKPT_DIR}/*
!rm {CONFIG_DIR}pipeline.config
!curl {PRETRAINED_MODEL_CONFIG} -o {CONFIG_DIR}pipeline.config

## Start the training


In [ ]:
%cd /content
!python /content/models/research/object_detection/model_main.py \
    --alsologtostderr \
    --pipeline_config_path={CONFIG_DIR}pipeline.config \
    --model_dir={CKPT_DIR} \
    --sample_1_of_n_eval_examples=1 \
    --sample_1_of_n_eval_on_train_examples=5
    

### Export TF Lite SSD Graph

In [ ]:
!python {BASE_DIR}/models/research/object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path={CONFIG_DIR}/pipeline.config \
--trained_checkpoint_prefix={CKPT_DIR}/model.ckpt-246 \
--output_directory={TF_LITE_GRAPH} \
--add_postprocessing_op=true

### Convert Graph to TF Lite Model

In [ ]:
!tflite_convert \
--output_file="{TF_LITE_MODEL}/detect.tflite" \
--graph_def_file="{TF_LITE_GRAPH}/tflite_graph.pb" \
--inference_type=QUANTIZED_UINT8 \
--input_arrays="{INPUT_TENSORS}" \
--output_arrays="{OUTPUT_TENSORS}"  \
--mean_values=128 \
--std_dev_values=128 \
--input_shapes=1,300,300,3 \
--change_concat_input_ranges=false \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops \
--output_format=TFLITE \
--default_ranges_min=0 \
--default_ranges_max=255

# Save 

## Model checkpoints

In [ ]:
!zip -r model_checkpointsv3.zip {CKPT_DIR}
!gsutil cp model_checkpointsv3.zip {DRIVE_DEV}

### TF Lite Model

In [ ]:
!zip -r part5-tflite-modelv7.zip {TF_LITE_MODEL}
!gsutil cp part5-tflite-modelv7.zip {DRIVE_DEV}